# Classification Metrics - Confusion Matrix, ROC/PR, Calibration, and Business Costs

<hr>

<center>
<div>
<img src="https://raw.githubusercontent.com/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/main/notebooks/figures/mgmt_474_ai_logo_02-modified.png" width="200"/>
</div>
</center>

# <center><a class="tocSkip"></center>
# <center>MGMT47400 Predictive Analytics</center>
# <center>Professor: Davi Moreira </center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/07_classification_metrics_thresholding.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Compute and interpret precision, recall, F1, ROC-AUC, PR-AUC
2. Select thresholds based on business cost tradeoffs
3. Understand calibration and when it matters
4. Handle class imbalance at the evaluation level (metrics first)
5. Produce a metrics dashboard table for model comparison

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score,
    ConfusionMatrixDisplay, RocCurveDisplay, PrecisionRecallDisplay
)
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 474
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

**Reading the output:**

The setup cell imports every metric function we will use in this notebook: `confusion_matrix`, `precision_score`, `recall_score`, `f1_score`, `roc_auc_score`, `roc_curve`, `precision_recall_curve`, and `average_precision_score`. It also imports the display helpers `ConfusionMatrixDisplay`, `RocCurveDisplay`, and `PrecisionRecallDisplay` that produce publication-quality plots with a single call. The `StandardScaler` and `LogisticRegression` imports let us build our baseline classification pipeline. The confirmation **"Setup complete!"** means all libraries loaded without conflict.

**Why this matters:** Having all metric functions imported up front keeps the rest of the notebook clean -- we can focus on *interpreting* results rather than wrestling with imports.

---

## 1. Load Data and Train Model

We begin with the **breast cancer Wisconsin dataset** (569 samples, 30 numeric features), a classic binary-classification benchmark packaged with scikit-learn. Each sample is labeled *malignant* (0) or *benign* (1) based on digitized measurements of cell nuclei from fine-needle aspirate images.

Before we can study classification metrics, we need predictions to evaluate. The cell below loads the data, applies the standard 60/20/20 train-validation-test split, and fits a Logistic Regression pipeline with `StandardScaler` as a preprocessing step. We then extract the confusion-matrix components (TP, TN, FP, FN) from the validation predictions so we can calculate every metric by hand in the sections that follow.

> 💡 **Gemini Prompt:** "Load the breast cancer dataset, split it 60/20/20 with stratification, then build and fit a logistic regression pipeline (StandardScaler + LogisticRegression with max_iter=1000). Get hard predictions and class-1 probabilities on the validation set. Print set sizes and validation accuracy."
>
> **After running, verify:**
> - Train/Val/Test sizes are printed
> - Validation accuracy is reported (should be above 95%)
> - Both y_pred_val (hard labels) and y_proba_val (probabilities) are available

In [ ]:
# Load dataset
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

# Split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=RANDOM_SEED, stratify=y_temp)

# Train model
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
])

pipeline.fit(X_train, y_train)
y_pred_val = pipeline.predict(X_val)
y_proba_val = pipeline.predict_proba(X_val)[:, 1]

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Test: {len(X_test)}")
print(f"Validation Accuracy: {pipeline.score(X_val, y_val):.4f}")

**Reading the output:**

The dataset has been split into **Train: 341 | Val: 114 | Test: 114** observations, following the 60/20/20 convention. The Logistic Regression pipeline (with `StandardScaler`) achieves a **validation accuracy around 0.97**, which sounds high but hides important nuances we will uncover below.

The variables `y_pred_val` (hard 0/1 labels) and `y_proba_val` (continuous probabilities for the positive class) are now available. Hard labels let us build a confusion matrix; probabilities let us sweep thresholds and plot ROC/PR curves.

**Key takeaway:** A single accuracy number is only the starting point. The next sections decompose this performance into the four cells of the confusion matrix and the metrics that derive from them.

---

## 2. The Confusion Matrix Deep Dive

### Understanding the 2×2 Matrix

```
                    Predicted Negative    Predicted Positive
Actual Negative          TN                     FP
Actual Positive          FN                     TP
```

**Definitions:**
- **True Positive (TP)**: Correctly predicted positive
- **True Negative (TN)**: Correctly predicted negative
- **False Positive (FP)**: Incorrectly predicted positive (Type I error)
- **False Negative (FN)**: Incorrectly predicted negative (Type II error)

> 💡 **Gemini Prompt:** "Compute the confusion matrix for validation predictions and display it as a heatmap using ConfusionMatrixDisplay with labels Malignant (0) and Benign (1). Unpack TN, FP, FN, TP using cm.ravel() and print each value."
>
> **After running, verify:**
> - Heatmap shows a 2x2 confusion matrix with labeled axes
> - TN, FP, FN, TP are printed as separate named values
> - Most predictions should be on the diagonal (correct classifications)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_val, y_pred_val)

fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Malignant (0)', 'Benign (1)'])
disp.plot(ax=ax, cmap='Blues', values_format='d')
plt.title('Confusion Matrix - Validation Set')
plt.tight_layout()
plt.show()

TN, FP, FN, TP = cm.ravel()
print("=== CONFUSION MATRIX VALUES ===")
print(f"True Negatives (TN):  {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"True Positives (TP):  {TP}")

**Reading the output:**

The heatmap shows the 2x2 confusion matrix on the validation set. The rows represent the *actual* class and the columns represent the *predicted* class. Darker blue squares indicate higher counts.

Below the plot, the four raw counts are printed:

- **True Negatives (TN):** correctly identified malignant cases.
- **False Positives (FP):** malignant cases the model mistakenly called benign -- these patients might be sent home without treatment.
- **False Negatives (FN):** benign cases the model mistakenly called malignant -- these patients would receive unnecessary follow-up.
- **True Positives (TP):** correctly identified benign cases.

Because the breast cancer dataset labels benign as 1 (positive class), a *False Negative* here means the model predicted malignant when the truth was benign. In a real clinical setting you would typically flip the positive class so that "malignant = positive," but for this educational exercise we follow scikit-learn's convention.

**Why this matters:** Every classification metric is just arithmetic on TN, FP, FN, and TP. Understanding these four numbers is the foundation for everything that follows.

---

## 3. Core Classification Metrics

### Precision
**Precision = TP / (TP + FP)**
- "Of all positive predictions, how many were correct?"
- High precision = few false alarms

### Recall (Sensitivity, True Positive Rate)
**Recall = TP / (TP + FN)**
- "Of all actual positives, how many did we find?"
- High recall = few missed positives

### F1 Score
**F1 = 2 × (Precision × Recall) / (Precision + Recall)**
- Harmonic mean of precision and recall
- Useful when you need balance

### Accuracy
**Accuracy = (TP + TN) / (TP + TN + FP + FN)**
- Overall correctness
- ⚠️ Misleading with class imbalance!

> 💡 **Gemini Prompt:** "Calculate classification metrics manually from TP, TN, FP, FN: Accuracy, Precision, Recall, F1 (using sklearn f1_score), and Specificity. Print all five metrics. Then print sklearn full classification_report with target names Malignant and Benign."
>
> **After running, verify:**
> - Five metrics are printed: Accuracy, Precision, Recall, F1, Specificity
> - classification_report shows per-class precision, recall, f1, and support
> - Manual calculations match sklearn automated values

In [ ]:
# Calculate all metrics
metrics = {
    'Accuracy': (TP + TN) / (TP + TN + FP + FN),
    'Precision': TP / (TP + FP) if (TP + FP) > 0 else 0,
    'Recall': TP / (TP + FN) if (TP + FN) > 0 else 0,
    'F1': f1_score(y_val, y_pred_val),
    'Specificity': TN / (TN + FP) if (TN + FP) > 0 else 0
}

print("=== CLASSIFICATION METRICS ===")
for metric, value in metrics.items():
    print(f"{metric:15s}: {value:.4f}")

# Using sklearn functions
print("\n=== SKLEARN CLASSIFICATION REPORT ===")
print(classification_report(y_val, y_pred_val, target_names=['Malignant', 'Benign']))

**Reading the output:**

The first block prints five hand-calculated metrics:

| Metric | Typical value | What it answers |
|--------|--------------|----------------|
| **Accuracy** | ~0.97 | What fraction of all predictions were correct? |
| **Precision** | ~0.97-0.99 | When we predicted benign, how often were we right? |
| **Recall** | ~0.97-0.99 | Of all truly benign cases, how many did we catch? |
| **F1** | ~0.98 | Harmonic mean balancing precision and recall |
| **Specificity** | ~0.93-0.97 | Of all truly malignant cases, how many did we flag? |

The second block, `classification_report`, confirms these numbers and adds the per-class breakdown (Malignant row and Benign row) plus macro and weighted averages. The `support` column shows how many validation samples belong to each class.

**Key takeaway:** Precision and recall often tell different stories. In a medical context, missing a malignant tumor (low specificity) is far worse than an unnecessary biopsy (low precision for the malignant class). The right metric depends on the *business cost* of each error type, which we explore in the PAUSE-AND-DO exercise.

---

## 4. ROC Curve and AUC

### ROC (Receiver Operating Characteristic)
- Plots True Positive Rate vs False Positive Rate
- Shows performance across all thresholds
- AUC (Area Under Curve) summarizes performance

**AUC Interpretation:**
- AUC = 1.0: Perfect classifier
- AUC = 0.5: Random guessing
- AUC < 0.5: Worse than random (predictions inverted)

> 💡 **Gemini Prompt:** "Plot the ROC curve using sklearn roc_curve on validation probabilities. Include the diagonal random-classifier baseline as a dashed line. Compute and display ROC-AUC score in the legend. Label axes as False Positive Rate and True Positive Rate."
>
> **After running, verify:**
> - ROC curve is plotted with AUC value shown in the legend
> - Diagonal dashed line represents a random classifier (AUC=0.5)
> - ROC-AUC score is printed below the plot (should be close to 1.0)

In [ ]:
# ROC Curve
fpr, tpr, thresholds_roc = roc_curve(y_val, y_proba_val)
roc_auc = roc_auc_score(y_val, y_proba_val)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"ROC-AUC Score: {roc_auc:.4f}")
print("\n💡 Higher AUC = better separation between classes")

**Reading the output:**

The ROC curve plots the **True Positive Rate** (recall) on the y-axis against the **False Positive Rate** (1 - specificity) on the x-axis as we sweep the decision threshold from 1.0 down to 0.0. A perfect classifier hugs the top-left corner; the dashed diagonal represents a random coin flip.

The **AUC (Area Under the Curve)** is printed below the plot. For our Logistic Regression on the breast cancer validation set, expect an **ROC-AUC around 0.99**, indicating near-perfect discrimination between malignant and benign samples. An AUC of 0.99 means that if you randomly pick one benign and one malignant case, the model assigns a higher probability to the benign case 99% of the time.

**Why this matters:** ROC-AUC is a threshold-independent summary of model quality. It is especially useful during model *selection* because it evaluates the full range of operating points, not just the default 0.5 threshold.

---

## 5. Precision-Recall Curve

### When to Use PR Curve vs ROC

**Use ROC when:**
- Classes are balanced
- You care about both TPR and FPR

**Use PR Curve when:**
- Classes are imbalanced
- Positive class is more important
- You want to focus on precision/recall tradeoff

> 💡 **Gemini Prompt:** "Plot the Precision-Recall curve using sklearn precision_recall_curve. Add a horizontal baseline at the positive class ratio. Compute average_precision_score and show it in the legend. Label axes as Recall (x) and Precision (y)."
>
> **After running, verify:**
> - PR curve is plotted with Average Precision (AP) shown in the legend
> - Horizontal dashed baseline shows the positive class ratio
> - AP score is printed below the plot

In [ ]:
# Precision-Recall Curve
precision, recall, thresholds_pr = precision_recall_curve(y_val, y_proba_val)
pr_auc = average_precision_score(y_val, y_proba_val)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, linewidth=2, label=f'PR Curve (AP = {pr_auc:.3f})')
baseline = y_val.sum() / len(y_val)
plt.axhline(y=baseline, color='k', linestyle='--', linewidth=1, label=f'Baseline (class ratio = {baseline:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average Precision (PR-AUC): {pr_auc:.4f}")
print("\n💡 Closer to top-right = better model")

**Reading the output:**

The Precision-Recall curve plots **Precision** on the y-axis against **Recall** on the x-axis. Unlike the ROC curve, the baseline here is the *class prevalence* -- the dashed horizontal line at the positive-class ratio (roughly 0.63 for benign in this dataset). A model that predicts the majority class for every sample would land on that baseline.

The **Average Precision (AP)** score, also called PR-AUC, summarizes the area under this curve. For our model, expect **AP around 0.99**, mirroring the strong ROC-AUC. The curve stays close to the top-right corner, meaning the model maintains high precision even at high recall levels.

**Key takeaway:** PR curves become more informative than ROC curves when classes are *imbalanced*. If only 5% of samples are positive, ROC-AUC can look optimistic because TN counts dominate the False Positive Rate denominator. PR-AUC focuses exclusively on the positive class, making it the metric of choice in fraud detection, rare-disease screening, and similar domains.

---

## 📝 PAUSE-AND-DO Exercise 1 (5 minutes)

**Task:** Build a threshold sweep and pick a threshold by business cost.

**Scenario:** Medical diagnosis
- False Negative (missed cancer): Cost = $50,000 (late treatment)
- False Positive (false alarm): Cost = $1,000 (unnecessary biopsy)

---

> 💡 **Gemini Prompt:** "Sweep 50 thresholds from 0.1 to 0.9. For each, compute TP, FP, FN, TN, Precision, Recall, and total cost using cost_FN=50000 and cost_FP=1000. Find the threshold that minimizes total cost. Plot two side-by-side charts: (1) Total Cost vs Threshold with the optimal threshold marked, and (2) Precision and Recall vs Threshold."
>
> **After running, verify:**
> - Optimal threshold and its expected cost are printed
> - Left plot shows a cost curve with the minimum marked by a red dashed line
> - Right plot shows precision increasing and recall decreasing as threshold rises

In [ ]:
# Threshold sweep with cost analysis
cost_FN = 50000  # Cost of missing a malignant case
cost_FP = 1000   # Cost of false alarm

thresholds_to_test = np.linspace(0.1, 0.9, 50)
threshold_results = []

for thresh in thresholds_to_test:
    y_pred_thresh = (y_proba_val >= thresh).astype(int)
    cm = confusion_matrix(y_val, y_pred_thresh)
    TN, FP, FN, TP = cm.ravel()
    
    # Calculate metrics
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    
    # Calculate expected cost
    total_cost = (FN * cost_FN) + (FP * cost_FP)
    
    threshold_results.append({
        'Threshold': thresh,
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN,
        'Precision': precision,
        'Recall': recall,
        'Total_Cost': total_cost
    })

threshold_df = pd.DataFrame(threshold_results)

# Find optimal threshold (minimum cost)
optimal_idx = threshold_df['Total_Cost'].idxmin()
optimal_thresh = threshold_df.loc[optimal_idx, 'Threshold']
optimal_cost = threshold_df.loc[optimal_idx, 'Total_Cost']

print("=== OPTIMAL THRESHOLD BY COST ===")
print(f"Optimal Threshold: {optimal_thresh:.3f}")
print(f"Expected Cost: ${optimal_cost:,.0f}")
print(f"\nAt this threshold:")
print(f"  Precision: {threshold_df.loc[optimal_idx, 'Precision']:.4f}")
print(f"  Recall: {threshold_df.loc[optimal_idx, 'Recall']:.4f}")
print(f"  FP: {threshold_df.loc[optimal_idx, 'FP']:.0f}")
print(f"  FN: {threshold_df.loc[optimal_idx, 'FN']:.0f}")

# Visualize cost vs threshold
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(threshold_df['Threshold'], threshold_df['Total_Cost'], linewidth=2)
axes[0].axvline(x=optimal_thresh, color='r', linestyle='--', label=f'Optimal = {optimal_thresh:.3f}')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Expected Cost ($)')
axes[0].set_title('Total Cost vs Threshold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(threshold_df['Threshold'], threshold_df['Precision'], label='Precision', linewidth=2)
axes[1].plot(threshold_df['Threshold'], threshold_df['Recall'], label='Recall', linewidth=2)
axes[1].axvline(x=optimal_thresh, color='r', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('Score')
axes[1].set_title('Precision-Recall vs Threshold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Reading the output:**

The output reports the **optimal threshold** that minimizes expected business cost. With a False Negative cost of **$50,000** (missed malignant case) and a False Positive cost of **$1,000** (unnecessary biopsy), the optimizer typically picks a threshold well below 0.5 -- sometimes around **0.1-0.3** -- because the penalty for missing cancer far outweighs the cost of a false alarm.

At the optimal threshold you will see:
- **Recall near 1.0:** The model catches nearly every malignant case.
- **Precision drops:** More false positives are tolerated.
- **Total cost minimized:** The combined dollar cost across all validation samples is as low as possible.

The left plot shows the U-shaped cost curve -- cost is high when the threshold is too strict (many FN) and high again when it is too lenient (many FP), with a sweet spot in between. The right plot shows precision and recall as functions of threshold, illustrating the classic tradeoff: as threshold decreases, recall rises but precision falls.

**Why this matters:** The default 0.5 threshold is arbitrary. Real-world deployment decisions should be driven by the *business cost structure*, not by a mathematical convenience. This cost-based threshold selection is a technique you will use in your course project.

---

### YOUR ANALYSIS:

**Question 1: Why is the optimal threshold different from 0.5?**  
[Your answer - think about asymmetric costs]

**Question 2: What happens if you change the cost ratio?**  
[Your answer - try FN cost = $100,000]

**Question 3: In production, how would you monitor this?**  
[Your answer - what could go wrong over time?]

---

## 📝 PAUSE-AND-DO Exercise 2 (5 minutes)

**Task:** Explain why accuracy fails under imbalance (with evidence).

---

> 💡 **Gemini Prompt:** "Create a synthetic imbalanced dataset with make_classification: 1000 samples, 20 features, 95%/5% class split. Split 70/30 with stratification, fit a logistic regression pipeline, and print the classification_report. Compare against a naive all-zeros baseline to show that high accuracy can be misleading with class imbalance."
>
> **After running, verify:**
> - Class distribution confirms ~95% class 0 and ~5% class 1
> - classification_report shows per-class metrics (class 1 recall may be low)
> - Naive baseline accuracy is ~95% despite predicting all zeros (zero recall for class 1)

In [ ]:
# Create imbalanced scenario
# Simulate: 95% benign, 5% malignant
from sklearn.datasets import make_classification

X_imb, y_imb = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    weights=[0.95, 0.05],  # 95% class 0, 5% class 1
    random_state=RANDOM_SEED
)

X_train_imb, X_val_imb, y_train_imb, y_val_imb = train_test_split(
    X_imb, y_imb, test_size=0.3, random_state=RANDOM_SEED, stratify=y_imb
)

# Train model
pipe_imb = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
])
pipe_imb.fit(X_train_imb, y_train_imb)

y_pred_imb = pipe_imb.predict(X_val_imb)

# Compare metrics
print("=== IMBALANCED DATASET (95% class 0, 5% class 1) ===")
print(f"\nClass distribution:")
print(pd.Series(y_val_imb).value_counts(normalize=True))

print(f"\n=== MODEL PERFORMANCE ===")
print(classification_report(y_val_imb, y_pred_imb, target_names=['Class 0 (95%)', 'Class 1 (5%)']))

# Naive baseline: always predict majority class
y_naive = np.zeros_like(y_val_imb)
naive_acc = (y_naive == y_val_imb).mean()

print(f"\n=== NAIVE BASELINE (always predict 0) ===")
print(f"Accuracy: {naive_acc:.4f}")
print(f"Recall for class 1: 0.0000 (missed all positives!)")

print(f"\n💡 Key Insight: {naive_acc*100:.1f}% accuracy by predicting everything as class 0!")
print(f"💡 This is why accuracy alone is dangerous with imbalance")

**Reading the output:**

This cell creates a synthetic dataset with **95% class 0 and 5% class 1** using `make_classification`, then trains a Logistic Regression and compares it against a naive baseline that *always predicts class 0*.

Key findings:
- The **naive baseline achieves ~95% accuracy** by always predicting the majority class -- despite being completely useless for identifying the minority class.
- The `classification_report` for the trained model shows that while overall accuracy looks respectable, **recall for class 1 may be significantly lower** than for class 0, because the model has very few positive examples to learn from.
- The naive baseline has **recall = 0.0** for class 1, meaning it misses every single positive case.

This is the **accuracy paradox**: a metric that appears excellent on the surface but hides total failure on the class you actually care about.

**Key takeaway:** When classes are imbalanced, always supplement accuracy with precision, recall, F1, and AUC. In extreme cases (fraud, rare disease), accuracy should be *ignored entirely* in favor of PR-AUC and recall.

---

### YOUR EXPLANATION:

**Why accuracy is misleading:**  
[Your explanation with evidence from above]

**Better metrics for imbalance:**  
[Which metrics would you use instead?]

**Real-world example:**  
[Give an example where this matters]

---

## 6. Metrics Dashboard

Individual metrics tell part of the story; a **dashboard** tells the whole story at a glance. The function below wraps every metric we have discussed -- accuracy, precision, recall, F1, specificity, ROC-AUC, and PR-AUC -- into a single call that returns a dictionary you can log, compare across models, or embed in a report.

Building a reusable dashboard function is a best practice you will carry into your course project. It guarantees that every model you evaluate is measured on the same set of metrics, making apples-to-apples comparison straightforward.

> 💡 **Gemini Prompt:** "Define a function create_metrics_dashboard(y_true, y_pred, y_proba) that computes Accuracy, Precision, Recall, F1, Specificity, ROC-AUC, PR-AUC, and the four confusion matrix values (TP, FP, FN, TN), returning them as a dictionary. Call it on the validation set and print all metrics in a formatted dashboard."
>
> **After running, verify:**
> - Function returns a dictionary with 11 keys (7 metrics + 4 confusion matrix counts)
> - All performance metrics are printed in a clean formatted layout
> - ROC-AUC and PR-AUC are included alongside threshold-dependent metrics

In [ ]:
# Complete metrics dashboard
def create_metrics_dashboard(y_true, y_pred, y_proba):
    """Generate comprehensive classification metrics"""
    cm = confusion_matrix(y_true, y_pred)
    TN, FP, FN, TP = cm.ravel()
    
    metrics = {
        'Accuracy': (TP + TN) / (TP + TN + FP + FN),
        'Precision': TP / (TP + FP) if (TP + FP) > 0 else 0,
        'Recall': TP / (TP + FN) if (TP + FN) > 0 else 0,
        'F1': f1_score(y_true, y_pred),
        'Specificity': TN / (TN + FP) if (TN + FP) > 0 else 0,
        'ROC_AUC': roc_auc_score(y_true, y_proba),
        'PR_AUC': average_precision_score(y_true, y_proba),
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN
    }
    return metrics

dashboard = create_metrics_dashboard(y_val, y_pred_val, y_proba_val)

print("=== COMPREHENSIVE METRICS DASHBOARD ===")
print(f"\nPerformance Metrics:")
print(f"  Accuracy:    {dashboard['Accuracy']:.4f}")
print(f"  Precision:   {dashboard['Precision']:.4f}")
print(f"  Recall:      {dashboard['Recall']:.4f}")
print(f"  F1 Score:    {dashboard['F1']:.4f}")
print(f"  Specificity: {dashboard['Specificity']:.4f}")
print(f"  ROC-AUC:     {dashboard['ROC_AUC']:.4f}")
print(f"  PR-AUC:      {dashboard['PR_AUC']:.4f}")

print(f"\nConfusion Matrix:")
print(f"  TP: {dashboard['TP']:4d}    FP: {dashboard['FP']:4d}")
print(f"  FN: {dashboard['FN']:4d}    TN: {dashboard['TN']:4d}")

**Reading the output:**

The dashboard function `create_metrics_dashboard` returns a dictionary with every metric in one place. The printed output is organized into two sections:

1. **Performance Metrics:** Accuracy, Precision, Recall, F1, Specificity, ROC-AUC, and PR-AUC -- all computed on the original breast cancer validation set. Expect values in the **0.95-0.99** range for this well-separated dataset.
2. **Confusion Matrix counts:** TP, FP, FN, TN as raw integers so you can verify any metric by hand.

This function is designed to be *reusable*: you can call `create_metrics_dashboard(y_true, y_pred, y_proba)` for any model and get a standardized report. In your course project, you will call this function for each candidate model and collect the results into a comparison DataFrame.

**Why this matters:** A metrics dashboard eliminates the risk of forgetting a metric or computing it inconsistently. It is the final step before presenting results to stakeholders, and it ensures that every model is evaluated on exactly the same terms.

---

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Confusion Matrix**: Foundation for understanding classification errors
2. **Precision vs Recall**: Tradeoff between false positives and false negatives
3. **ROC and PR Curves**: Visualize performance across thresholds
4. **Cost-Based Thresholding**: Align decisions to business objectives
5. **Imbalance Handling**: Accuracy is dangerous - use precision, recall, AUC

### Critical Rules:

> **"Never trust accuracy alone"**

> **"Choose thresholds based on business costs, not defaults"**

> **"With imbalance, use PR curves over ROC curves"**

### Next Steps:

- Next notebook: Cross-validation for robust model comparison
- We'll use today's metrics with proper CV evaluation
- Apply to your project dataset

---

## Participation Assignment Submission Instructions

### To Submit This Notebook:

1. **Complete all exercises**: Fill in both PAUSE-AND-DO exercise cells with your findings
2. **Run All Cells**: Execute `Runtime → Run all` to ensure everything works
3. **Save a Copy**: `File → Save a copy in Drive or Download the .ipynb extension`
4. **Submit**: Upload your `.ipynb` file in the participation assignment you find in the course Brightspace page.

### Before Submitting, Check:

- [ ] All cells execute without errors
- [ ] All outputs are visible
- [ ] Both exercise responses are complete
- [ ] Notebook is shared with correct permissions
- [ ] You can explain every line of code you wrote

### Next Step:

Complete the **Quiz** in Brightspace (auto-graded)

---

## Bibliography

- Fawcett, T. (2006). "An introduction to ROC analysis." *Pattern Recognition Letters*, 27(8), 861-874.
- Saito, T., & Rehmsmeier, M. (2015). "The precision-recall plot is more informative than the ROC plot when evaluating binary classifiers on imbalanced datasets." *PLOS ONE*.
- scikit-learn User Guide: [Classification metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)
- Provost, F., & Fawcett, T. (2013). *Data Science for Business* - Chapter on evaluation and costs

---



<center>

Thank you!

</center>